# INF 8215 - Intelligence artif.: méthodes et algorithmes 
## Fall 2018 - TP1 - Research Methods 
### Team Components
    - Boucenna Manal Maroua
    - Masson Matthieu
    - Michelon Paul



## Biking in Montreal
Approximately 10 million tourists visit Montreal every year. Concerned in providing the best experience during their stay, the tourism department of the city has initialized the development of a new application to help the visitors meet the main touristic attractions of the city in the shortest amount of time. They discovered that, when visiting the city’s attractions, the preferred mean of transportation adopted by the tourists is the bicycle. However, since the tourists do not know the city very well, they usually struggle to create their route in order to visit the planned sights for the day. Thus, given a list of places that the tourist wants to visit, including a starting and ending location, the new application determines the itinerary that passes only once in all attractions defined by the user such that the time spent within his/her route is minimum. 

<img src="images/montreal.png" alt="" width="800"/>

Our job in this TP is to develop this new application. To this end, we will address the problem using three different research methods:
1. The first methodology works by defining and exploring a search tree in order to find the route with minimum time
2. Next, an A* algorithm will be designed to search the optimal route in this search tree more efficiently 
3. Last, a heuristic procedure will be created using the Variable Neighborhood Search (VNS) metaheuristic to optimize the problem

## Problem representation

We can represent our problem using a directed and complete graph, $G(V, A)$, where each city attraction is represented by a vertex in the set of vertices $V$, and $A$ is the set of arcs representing the bicycle path between two attractions. Each arc $a_{ij} \in A$ has a cost $ w(a_{ij}) $ associated with the time spent on the path from attraction $i$ to attraction $j$. However, it is worth mentioning that the times are not symmetric, hence, is **not** guaranteed that $w(a_{ij}) = w(a_{ji})$. 

The user of the tool informs the list of $m$ attractions that he/she wishes to visit, $P = \{p_1,...,p_m\} $, where $p_1$ and $p_m$ are the starting and ending attractions, respectively.

## 1. Defining and exploring a search tree

Let us define a search tree, $\mathcal{T}$, where each node of this tree represents a solution $S$ that is being built. Moreover, let $V(S) \subseteq V$ be the set of vertices already **visited** and $A(S) \subset A$ the set of arcs already **selected**. Thus, the cost of $S$ is given by

$$g(S) = \sum_{a \in A(S)} w(a)$$

The root node of the search tree $\mathcal{T}$ is a solution $S_{root}$ containing only the starting vertex marked as visited, i.e., $V(S_{root})=\{p_1\}$ and $A(S_{root}) = \emptyset$.

<img src="images/tree1.png" alt="" width="100"/>

Then, children nodes are created by adding an arc from $p_1$ to every vertex $ v \in P\backslash V(S_{root})$. However, an arc to the ending attraction can only be added if $p_m$ is the only vertex not visited yet. We denote the new visited vertex as $c$, the current vertex of the solution, i.e., $V(S) \gets V(S) \cup \{c\}$. 

<img src="images/tree2.png" alt="" width="400"/>

Next, each child node similarly generates its children. For a given child solution $S$ with last added vertex $c$, we add an arc $(c, v)$ to every vertex $v \in P\backslash V(S)$. The constraint to add vertex $p_m$ to the solution is kept during the entire process.

<img src="images/tree3.png" alt="" width="600"/>

The leaf nodes are complete solutions, i.e., $V(S) = P$, with $p_m$ being the last attraction added to the itinerary.  

### 1.1 Time to code

The code cell below is provided to you with a function to read the graph representing the attractions of Montreal. The function $\mathtt{read\_graph}$ returns a numpy array object $|V|\times |V|$ where the entry $\mathtt{graph[i,j]}$ contains the time spent biking from attraction $i$ to attraction $j$.



In [1]:
import numpy as np

def read_graph():
    return np.loadtxt("montreal", dtype='i', delimiter=',')

graph = read_graph()

Our first implementation task is to code the class that represents a solution. The code of class Solution below has a constructor (already implemented) that receives the list of $\mathtt{places}$ representing the set of attractions $P$ and the graph $G$, and creates a blank solution by initializing the following variables:

- $\mathtt{g}$: stores the cost of the solution
- $\mathtt{c}$: indicate the index of the current (last added) vertex of the solution
- $\mathtt{V}$: a list representing $V(S)$
- $\mathtt{unvisited}$: a list representing $P\backslash V(S)$

Alongside the constructor, there is the method $\mathtt{add}$ which is responsible for adding an arc into the solution. This method receives the index from the $\mathtt{unvisited}$ list indicating the attraction that will be added. This method also updates the $\mathtt{V}$ and $\mathtt{unvisited}$ lists, as well the cost of the solution and vertex $c$.

Now, implement the method $\mathtt{add}$ below.

In [2]:
import copy

class Solution:
    def __init__(self, places, graph):
        """
        places: a list containing the indices of attractions to visit
        p1 = places[0]
        pm = places[-1]
        """
        self.h = 0  #heuristic
        self.g = 0  # cost
        self.f = 0
        self.graph = graph
        self.visited = [places[0]]  # list of already visited attractions
        self.not_visited = copy.deepcopy(places[1:])  # list of attractions not yet visited

    def add(self, idx):
        """
        Adds the point in position idx of not_visited list to the solution
        """
        self.visited.append(idx)
        self.not_visited.remove(idx)
        self.g = self.g + graph[self.visited[-2], idx]

    def __lt__(self, other):
        return self.f < other.f

Our next step is to implement a research method to find the best solution in the search tree. A simple strategy to this end would be a [Breadth-first search](https://moodle.polymtl.ca/pluginfile.php/444662/mod_resource/content/1/recherche_en_largeur.mp4) (BFS), which explores the nodes in the sequence as they are scanned using a queue data structure. 

Now, implement the $\mathtt{bfs}$ method below that receives the graph and the list of attractions and returns the solution with minimum cost.  

In [19]:
from queue import Queue

def bfs(graph, places):
    """
        The BFS Algorithm works as follow:
            Step 1- We use a queue 'queue' to store our objects;
            Step 2- We take the last element of the queue 'current_node' while the queue is full :
                    2-a: if the its visited's last element is the goal => complete solution
                         We update the solution by adding the goal to its path and compare it to the other complete paths
                         if it has a smaller cost than the other complete solutions we update it in: Optimal_solution
                    2-b: ELSE visit all current_node'neighbours (not_visited) and update their paths and cost before putting 
                         them back to the queue
            STEP3- return the Solution with minimal cost
            
    """
    S_root = Solution(places,graph)
    # STEP 1
    queue = Queue()
    goal = places[-1]
    best_time = 1000000 # First cost
    queue.put(S_root)
    explore = 0
    while queue.full():
        # STEP 2
        current_node = queue.get()
        explore += 1
        # STEP 2-a
        if current_node.not_visited == [goal]:
                current_node.add(goal)
                if (current_node.g < best_time):
                    best_time = copy.deepcopy(current_node.g)
                    OptimalSolution = copy.deepcopy(current_node)
        # STEP 2-b
        else:
            for i in current_node.not_visited[:-1]:
                Temp=copy.deepcopy(current_node)
                Temp.add(i)
                queue.put(Temp)
    # STEP 3
    print("Number of nodes explored: %s" %(explore))            
    return OptimalSolution

### 1.2 Experiments

We propose three illustrative examples to test our search in width. The first example takes into account 7 attractions, the second 10 and the last 11. Since this research lists all possible solutions, the third example may take a considerable time to complete.

Implement these experiments and report the number of nodes explored as well as the required computing time.

In [30]:
import time 

#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

Number of nodes explored: 326
27
--- 0.031928300857543945 seconds ---


In [31]:
#test 2 -------------- OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

Number of nodes explored: 109601
30
--- 28.89189577102661 seconds ---


In [22]:
#test 3 -------------- OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

Number of nodes explored: 500055
26
--- 1030.7141654491425 seconds ---


## 2. Solving with an A* algorithm

For our second research method, instead of enumerating all solutions, we intend to guide the search by evaluating the potential of each node of the search tree, prioritizing the solutions with lower estimate costs. To this purpose, each solution $S$ is evaluated by the heuristic function $f(S) = g(S) + h(S) $ that takes into consideration both the current cost of $S$, $g(S)$, and an estimate of the cost (lower bound), $h(S)$, to visit the ending attraction $p_m$ from $S$.

For a given solution $S$, with $ c $ being the last added vertex to $V(S)$, the function $h$ could be defined as:

$$h(S) = \text{the shortest path from } c \text{ to } p_m \text{ in the graph } G_S  \text{ induced by the set of vertices } P\backslash V(S) \cup \{c\} $$

In other words, $h(s)$ is a lower bound to the time required to the tourist to arrive at the last attraction, $p_m$, from the current attraction, $c$, passing only through attractions of $P$ still not visited.


Hence, we can design our A* algorithm dividing it into the following steps:

1. As prevously done, the search tree $\mathcal{T}$ is initialized with a root solution $S_{root}$ that contains only the starting point, i.e., $V(S_{root}) = p_1, A(S_{root}) = \emptyset $ and $ c = p_1 $.  The cost of this initial solution is $ g(S_{root}) = 0 $. The estimative for this root solution is not important and can be discarded ($h(S_{root}) = 0 $).

2. Select the solution $S_b$ of  $\mathcal{T}$ that minimizes the heuristic function $f$:
$$ f(S_b) \leq f(S) \quad \forall S \in \mathcal{T} \qquad S_B, S \text{ are nodes of } \mathcal{T} \text{ not yet selected}$$
Note that, at the first iteration, only the $S_{root}$ is present in $\mathcal{T}$ and, thus, it will be selected. If $S_b$ is complete, then the optimal solution of the problem is found, and the algorithm ends. Otherwise, the algorithm proceeds normally to step 3.

3. Let $c$ be the last visited vertex in $S_b$. For each vertex $ v \in P\backslash V(S_b) $, perform the following tasks:
  - Create a new solution $S_n$ by adding the arc $ (c, v) $ to the solution
  - Update the sets $V(S_n)$ and $A(S_n)$;
  - Compute $g(S_n)$ and $h(S_n)$;
  - Insert $S_n$ as a new child node in $\mathcal{T}$
  
  The vertex $v = p_m$ can be only added to a new solution if it is the only vertex remaining in the set $ P\backslash V(S_b)$. 

4. Repeat steps 2 and 3 until the selected solution $S_b$ is **complete**, i.e., $V(S_B) = P$.

### 2.1 Time to code

Our first implementation task is to increment the code of the class Solution. As we shall see later, a priority queue (heap) is going to be used to implement the search tree T, instead of an ordinary queue previously used. Then, we must prepare the class Solution with a method able to compare two solutions. This is done with the method $\_\_\mathtt{lt}\_\_\mathtt{(self, other)}$ which must return true if $f(self) < f(other)$.

Now, implement the function $\_\_\mathtt{lt}\_\_\mathtt{(self, other)}$ in the class Solution.

Following that, the next step is to implement the estimation function. To compute the fastest path between two vertices of a graph we can use the [Dijkstra algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) where the source node is the last added vertex, $c$. The Dijkstra algorithm can be adapted to finish as soon as the fastest path to vertex $p_m$ is found.

**Implementation guidelines**:
  - Apply the Dijkstra algorithm to find the fastest path from $c$ to $p_m$.
  - Return the time spent in the path found.

In [7]:
import heapq

def fastest_path_estimation(sol):
    """
    The Djikstra algorithm works as follow:
        Step 1: Add in a heapq 'queue', the start point c and all the places to visit using the turple: (cost,node)
                We initiate the nodes' cost with a infinity or a big value, in our case 1000.
                queue = [(0,c),(1000p1),(1000, p2)...(1000,pm)]
        Step 2: Pop the last element of the list 'queue', the one with the smallest cost: current_node
                  2-a IF the element popped is the destination point 
                        return its cost, this means that we found the shortest path from c to pm (It doesn't need to 
                        visit all the places)
                  2-b ELSE visit all element of queue (actual_node) and compare their actual cost to current node'cost the + the cost
                           from current node to the actual node, we take the min of them.
                           To do this we used a heapq Temp as a transition queue
                        
    """
    SOL=copy.deepcopy(sol)
    c = SOL.visited[-1]
    pm = SOL.not_visited[-1]
    voisins = SOL.not_visited
    ####################################
    # STEP 1
    queue = []
    heapq.heapify(queue)
    heapq.heappush(queue, (0,c))
    for i in voisins:
        heapq.heappush(queue,(1000,i))
    ####################################
    Temp =[]
    heapq.heapify(Temp)
    while queue:
    ####################################
    # STEP 2
        current_node = heapq.heappop(queue)
    #STEP 2-a
        if  current_node[1]==pm:
            return current_node[0]
    #STEP 2-b
        while queue:
            actual_element=heapq.heappop(queue)
            heapq.heappush(Temp,(min(current_node[0]+graph[current_node[1],actual_element[1]],actual_element[0]),actual_element[1]))
        while Temp:
            heapq.heappush(queue,heapq.heappop(Temp))

Finally, we must implement the A* function that creates the solution tree and performs the search. To implement the search, we will use a priority queue data structure (heap) to automatically select the solution $S_b$ from $\mathcal{T}$.

**Implementation guidelines**:
  - While a complete solution is not selected from the search tree $\mathcal{T}$ do:
      - Selected the top solution of the heap.
      - Perform step 3 described above
  - Return the optimal solution obtained.

In [8]:
def A_star(graph, places):
    """
    The A* algorithm works as follow:
        STEP a: It pops the last element of the heapq 'T' in an Object called "current_node", then:
            a-1: IF the current_node has an empty 'visited' => it means all the solution is complete and the most optimal
            a-2: IF the current_nodehas completed its path we add the goal to the solution anf pop it to the heapq T
            a-3: ELSE the solution isn't complete, we add the non-visited places to the solutions and calculate their 
                    respective heuristic and cost and push them to the heapq that will sort its element depending on 
                    the value of f = g + h
    """
    # blank solution
    root = Solution(graph=graph, places=places)
    # search tree T
    T = []
    heapq.heapify(T)
    heapq.heappush(T, root)
    goal = places[-1]
    explore=0
    while T:
    # STEP a
        current_node = heapq.heappop(T)
        explore += 1
    #Step a-1
        if len(current_node.not_visited) == 0:
            print("Number of nodes explored: %s" % (explore))
            return current_node
    #Step a-2
        if current_node.not_visited == [goal]:
            current_node.add(goal)
            current_node.f = current_node.g
            heapq.heappush(T, current_node)
    #Step a-3
        else:
            for i in current_node.not_visited[:-1]: #pour tout i dans les element non visites (restants)
                Temp = copy.deepcopy(current_node)
                Temp.add(i)
                Temp.h = fastest_path_estimation(Temp)
                Temp.f = Temp.h + Temp.g
                heapq.heappush(T, Temp) #ajout a  la liste et ordonnner

### 2.2 Experiments

To test our A* algorithm, a fourth experiment with 15 attractions was included in the set of experiments. Once again, for each test, report the time of execution and the number of nodes explored in the search. 

In [28]:
#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

Number of nodes explored: 58
27
[0, 5, 13, 16, 6, 9, 4]
--- 0.019545555114746094 seconds ---


In [27]:
#test 2  --------------  OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

Number of nodes explored: 360
30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 0.1441335678100586 seconds ---


In [26]:
#test 3  --------------  OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

Number of nodes explored: 915
26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 0.3845224380493164 seconds ---


In [25]:
#test 4  --------------  OPT. SOL. = 40
start_time = time.time()
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
astar_sol = A_star(graph=graph, places=places)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

Number of nodes explored: 108401
40
[0, 3, 5, 13, 15, 18, 20, 16, 11, 12, 14, 9, 4, 2, 1]
--- 78.01531720161438 seconds ---


### 2.3 A tighter lower bound - BONUS?

While the estimated cost of the fastest path helped the $A*$ algorithm to solve problems with a larger number of attractions, this lower bound does not takes into consideration the cost of visiting the other vertices in $P\backslash V(S)$. A better estimation for our problem can be given by the **spanning arborescence of minimum weight** (this is the minimum spanning tree version for directed graphs) for the graph $G_S$ induced by the set of vertices $P\backslash V(S) \cup \{c\} $. This problem seeks to create a tree rooted in a defined vertex, $r$, that visits all vertices of the graph with minimum cost.

For our problem, one could define the root vertex as the current vertex of the solution, $r = c$, and compute the sum of costs required to reach the vertices $P\backslash V(S)$. A classical procedure to solve this problem is the [Edmonds's algorithm](https://en.wikipedia.org/wiki/Edmonds%27_algorithm). Our task now is to implement this algorithm and use it as a new estimation cost, $h(S)$. After that, re-run the experiments in Section 2.2 and report the gains, concerning both the execution times and the number of explored nodes.




In [ ]:
def minimum_spanning_arborescence(sol):
    """
    Returns the cost to reach the vertices in the unvisited list 
    """
    SOL= copy.deepcopy(sol)
    root=SOL.visited[-1]
    

## 2. Optimizing with a VNS heuristic

This time, instead of building a solution from scratch by gradually adding arcs to the solution,  our [Variable Neighborhood Search](https://en.wikipedia.org/wiki/Variable_neighborhood_search) (VNS) method will start from a complete initial solution and performs successively swaps on the order in which the attractions are visited to improve the cost of the solution. 

<img src="images/vns.png" alt="" width="800"/>

### 2.1 Time to code

Our first implementation task is to construct an initial solution to work with during the search. This solution must be complete, i.e., we must create an ordered set of visited vertex $V(S)$ such that it starts in $p_1$ and finishes in $p_m$. A simple approach to this end is to use a [Depth-First Seach (DFS)](https://moodle.polymtl.ca/pluginfile.php/445484/mod_resource/content/1/recherche_en_profondeur.mp4). The starting point is the vertex representing $p_1$ and, as soon as the ending point is reached after visiting all the others attractions of $P$, the path can be turned into an initial solution. To help in diversifying the search, the method to generate an initial solution can be randomized such that the VNS algorithm can start the search in different regions of the solution space. Thus, in the DFS function, the selection of the child to continue the search must be random.

Now. implement the **initial_sol** method to construct a random initial solution and uses the **dfs** function to find a path from $p_1$ to $p_m$.



**Implementation guidelines**:
  - Call the dfs function to find a complete path from  $p_1$ to $p_m$.
  - Create a root solution
  - Adjust the solution cost and the set V with the path found
  - Return the solution created



In [ ]:
from random import shuffle, randint

def initial_sol(graph, places):
    """
    Return a completed initial solution
    """

def dfs(...):
    """
    Performs a Depth-First Search
    """

A key feature of the VNS metaheuristic is the ability to define $k_{max}$ neighborhoods to diversify the search and escape from local optima. For our problem, a simple and efficient neighborhood structure could be neighborhoods based on **swapping** the order in which two points are visited within the route. Thus, given a solution $S$, with $V(S)$ being the ordered set in which the points were visited, a solution on the $k$-th neighborhood of $S$ can be obtained by selecting $k$ *random* pairs of vertices, different from $p_1$ and $p_m$, and swap their position in $V(S)$.

In the VNS framework, the procedure of generating a solution on the $k$-th neighborhood of $S$ is known as the **shaking** step. Thus, our task now is to implement this shaking method that receives a solution, the neighborhood index $k$ and the graph $G$ and returns a new solution on the $k$-th neighborhood of $S$. 

However, before implementing the shaking method below, we must first return to the implementation of class Solution and finish the **swap** method. This method is responsible for updating the position of the vertices being swapped in the $V(S)$ set and, consequentily, the solution cost. It receives the indices of the two selected vertices of $V(S)$ and the graph $G$.


**Implementation guidelines (shaking)**:
  - Select two indices, $i$ and $j$, such that $i \neq j$ and $i, j \in \{2...m-1\}$
  - Make a copy of the $sol$ and perfom the swap
  - Return the created solution

In [ ]:
def shaking(sol, k):
    """
    Returns a solution on the k-th neighrboohood of sol
    """

Another essential step in the VNS algorithm is to invoke a local search routine to improve the solution cost. For our problem, an efficient local search procedure would be to search for a pair of vertices, $i$ and $j$, such that their swapping position in $V(S)$ leads it to a reduction in the solution cost. This local search is known as 2-opt, and we are going to implement in this TP. 

Formally, consider a solution $S$ and the ordered set of vertices $V(S)$. For a vertex $i$, let $i'$ be the immediate successor of $i$ in the sequence $V(S)$. The 2-opt algorithm work as follows: for every  pair of non-consecutive vertices $i,j$, check if by swapping the position of vertices $i'$ and $j$ results in a improvement in the solution cost. If so, perform this swap. This process is repeated until there are no more profitable exchanges. The 2-opt heuristic is illustrated in the figure below. 


<img src="images/2opt.png" alt="" width="800"/>

<img src="images/2opt2.png" alt="" width="800"/>



Now implement the **local_search_2opt** fuction below. 



**Implementation guidelines**:
  - Check every pair of indices, $i = \{2,..,m-3\}$ and $j = \{i+2, m-1\}$ for a improvement in the solution cost
  - If the exchange is profitable, perform it
  - Repeat the process until no more profitable swap can be found

In [ ]:
def local_search_2opt(sol):
    """
    Apply 2-opt local search over sol
    """



Finally, our last implementation step is to code the VNS algorithm itself. The method receives an initial solution  and returns the best solution found during the search. Alongside the solution and the graph, the method also receives two other parameters: $k_{max} $, which defines the number of neighborhoods; and $t_{max}$, which sets the time limit to run the heuristic.


**Implementation guidelines**:
  - At each iteration of the VNS, generate a solution on the k-th neighrborhood of the current best solution and apply a local search on it.
  - If the new solution has a better cost than the current best solution, update the best solution with the new found solution. 
  - Repeat the process until the time limit is met

In [ ]:
def vns(sol, k_max, t_max):
    """
    Performs the VNS algorithm
    """

### 2.2 Experiments

Run the VNS on the following illustrative examples and report the obtained solutions:

In [ ]:
# test 1  --------------  OPT. SOL. = 27
places=[0, 5, 13, 16, 6, 9, 4]
sol = initial_sol(graph=graph, places=places)
start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#test 2  --------------  OPT. SOL. = 30
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# test 3  --------------  OPT. SOL. = 26
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# test 4  --------------  OPT. SOL. = 40
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

## 4. BONUS SECTION (1 point)

Explain in which situation each of the developed algorithms is more appropriate (hint: take into consideration the scalability of the problem)

1- BFS Algorithm is suitable for finding the smallest number of steps it takes to get to any given node from the source in an unweighted graph. Also we can use BFS if we know that our goal is somehow near to the root node, in other word it depends on the depth of our Tree. BFS algorithm is good for small tests like test 1.

2- The A* algorithm is a better approach because it uses a heurisitc to better choose the next move and limit the number of node to visite. This algorithm is mainly used in path finding.

3-